# Considering an ion-exclusion (Stern) layer: a two-surface model

The Poisson-Boltzmann single surface models treat the ions in the solvent as point charges and tends to overestimate their concentration in the proximities of charged surfaces. In reality, ions have a finite size thus there is a maximum number of them that can be close to a surface.

To reduce this effect, one can include a new surface in the simulation that surrounds the first surface. The new region is called ion-exclusion layer or Stern Layer.

## Direct Stern Formulation

The Stern layer is only supported in the `direct` formulation, and is activated when defining the `Simulation`. Under the hood, this calls a different `direc_stern` formulation, which is a variation of the `direct` formulation that simulates two surfaces:  the original solute surface and the new ion-exclusion layer surface. Let´s see how it works: first we import the module and create the basic solute object.

In [ ]:
import pbj
simulation = pbj.electrostatics.Simulation(formulation = 'direct', stern_layer=True)
protein = pbj.electrostatics.Solute('pqrs/1bpi.pqr', mesh_density = 1)

The surface mesh that interfaces the Stern layer with the solvent is only generated when the solute is added to te simulation. This mesh usually doesn't need to be as fine as the dielectric interface. By default, we set its density to half the dielectric interface density, but it can be modified with `stern_mesh_density_ratio`. 

In [ ]:
print(protein.stern_mesh_density_ratio)

Other parameters of the ion-exclusion (Stern) layer, like its permittivity or width, can also be modified. They default to the solvent permittivity and 2Å, respectively.

In [ ]:
print(protein.ep_stern)
print(protein.pb_formulation_stern_width)

Let's now add the protein to the simulation

In [ ]:
simulation.add_solute(protein)

The dielectric interface mesh looks like this:

In [ ]:
simulation.solutes[0].mesh.plot()

The direct stern formulation includes new parameters for the solute object:

The new mesh is stored as a Solute object in the stern_object attribute:

In [ ]:
simulation.solutes[0].stern_object.mesh.plot()

There are two preconditioners available for this formulation, but for direct it defaults to `block_diagonal`

In [ ]:
print(simulation.solutes[0].display_available_preconditioners())
print(simulation.solutes[0].pb_formulation_preconditioning_type)

And now, we calculate. If you're curious of the progress of your simulation, wou can activate a verbose mode with `pbj.electrostatics.simulation.bempp.api.enable_console_logging("info")`

In [ ]:
pbj.electrostatics.simulation.bempp.api.enable_console_logging("info")
simulation.calculate_solvation_energy()

Now the results attribute stores the electric potential and it's derivative in the new surface:

In [ ]:
simulation.solutes[0].results.keys()

In [ ]:
print(simulation.solutes[0].results["solvation_energy"])

In [ ]:
simulation.solutes[0].results['phi_stern'].plot()